In [ ]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

# 📌 Konstanter
G = 6.67430e-11  # Gravitasjonskonstanten, m^3 kg^-1 s^-2
M_moon = 7.342e22  # Månens masse, kg
R_moon = 1.737e6  # Månens radius, m
mu = G * M_moon  # Standard gravitasjonsparameter, m^3 s^-2

# 🌍 Orbitalparametere
r_max = R_moon + 500000  # Maksimal avstand fra månens sentrum, m
r_min = R_moon + 50000  # Minimal avstand fra månens sentrum, m
a = (r_max + r_min) / 2  # Semi-major axis, m
b = np.sqrt(r_max * r_min)  # Semi-minor axis, m
e = np.sqrt(1 - (b**2 / a**2))  # Eksentrisitet
i = np.radians(0)  # Inclination, radianer
Ω = np.radians(0)  # Right ascension of the ascending node, radianer
ω = np.radians(0)  # Argument of periapsis, radianer
ν = np.linspace(0, 2 * np.pi, 500)  # True anomaly, radianer

# 🚀 Beregn posisjoner i baneplanet
r = a * (1 - e**2) / (1 + e * np.cos(ν))  # Avstand fra månens sentrum
x_orbit = r * np.cos(ν)
y_orbit = r * np.sin(ν)
z_orbit = np.zeros_like(ν)

# 📌 Rotasjonsmatriser
R_Ω = np.array([
    [np.cos(Ω), -np.sin(Ω), 0],
    [np.sin(Ω), np.cos(Ω), 0],
    [0, 0, 1]
])

R_i = np.array([
    [1, 0, 0],
    [0, np.cos(i), -np.sin(i)],
    [0, np.sin(i), np.cos(i)]
])

R_ω = np.array([
    [np.cos(ω), -np.sin(ω), 0],
    [np.sin(ω), np.cos(ω), 0],
    [0, 0, 1]
])

# 🔄 Kombinert rotasjonsmatrise
R = R_Ω @ R_i @ R_ω

# 🎯 Roter baneplanet til 3D-rommet
orbit_3d = R @ np.vstack((x_orbit, y_orbit, z_orbit))

# 🚀 Beregn hastighetsvektoren i baneplanet
v = np.sqrt(mu / r)  # Absolutt fart (skalar)
vx = -v * np.sin(ν)  # Tangentiell fart
vy = v * np.cos(ν)   # Tangentiell fart
vz = np.zeros_like(ν)

# 🎯 Roter hastighetsvektoren til 3D-rommet
velocity_3d = R @ np.vstack((vx, vy, vz))

# 🔹 Beregn absolutt hastighet etter rotasjon
abs_velocity = np.sqrt(velocity_3d[0]**2 + velocity_3d[1]**2 + velocity_3d[2]**2)

# 🔹 Beregn høyde over månen
altitude = r - R_moon  # Høyde over måneoverflaten

# 🔹 Lag hovertekst med høyde og hastighet
hover_texts = [
    f"Høyde: {h:.2f} m<br>Hastighet: {v:.2f} m/s"
    for h, v in zip(altitude, abs_velocity)
]

# Beregn posisjonene og hastighetene for maksimal og minimal avstand
max_distance_index = np.argmax(r)
min_distance_index = np.argmin(r)
max_distance_point = orbit_3d[:, max_distance_index]
min_distance_point = orbit_3d[:, min_distance_index]
max_velocity = abs_velocity[max_distance_index]
min_velocity = abs_velocity[min_distance_index]

# 🌕 Generer sfæren for månen
phi, theta = np.mgrid[0:np.pi:400j, 0:2*np.pi:400j]
x_moon = R_moon * np.sin(phi) * np.cos(theta)
y_moon = R_moon * np.sin(phi) * np.sin(theta)
z_moon = R_moon * np.cos(phi)

# 🖼️ Last inn teksturbildet av månen
img = Image.open('moonmap4k.jpg').convert('L')  # Gråskala
img_resized = img.resize((400, 400))  # Tilpass til grid
texture = np.array(img_resized)


# 📊 Plot med Plotly
fig = go.Figure()

# 🚀 Legg til satellittbanen med hoverinfo for høyde og absolutt hastighet
fig.add_trace(go.Scatter3d(
    x=orbit_3d[0], y=orbit_3d[1], z=orbit_3d[2],
    mode='lines',
    line=dict(color='cyan', width=2),
    name='Satellittbane',
    text=hover_texts,
    hoverinfo='text'
))

# 🌕 Legg til månen med tekstur
fig.add_trace(go.Surface(
    x=x_moon, y=y_moon, z=z_moon,
    surfacecolor=texture, 
    colorscale='gray',
    opacity=1,
    showscale=False,
    name='Månen',
    hoverinfo='skip'
))

# Legg til punkter for maksimal og minimal avstand med hastighet
fig.add_trace(go.Scatter3d(
    x=[max_distance_point[0]], y=[max_distance_point[1]], z=[max_distance_point[2]],
    mode='markers+text',
    marker=dict(size=5, color='red'),
    text=[f"Høyde: {r_max - R_moon:.2f} m<br>Hastighet: {max_velocity:.2f} m/s"],
    hoverinfo='text',
    name='Maksimal avstand'
))

fig.add_trace(go.Scatter3d(
    x=[min_distance_point[0]], y=[min_distance_point[1]], z=[min_distance_point[2]],
    mode='markers+text',
    marker=dict(size=5, color='blue'),
    text=[f"Høyde: {r_min - R_moon:.2f} m<br>Hastighet: {min_velocity:.2f} m/s"],
    hoverinfo='text',
    name='Minimal avstand'
))

# 🎨 Layout og stil
fig.update_layout(
    title=dict(
        text='Satellittbane rundt månen med høyde og hastighet',
        font=dict(size=24, color='white'),
        x=0.5, y=0.95, xanchor='center', yanchor='top'
    ),
    width=800,
    height=600,
    scene=dict(
        xaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='x (m)'),
        yaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='y (m)'),
        zaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='z (m)'),
        bgcolor='black'
    ),
    legend=dict(
        x=0, y=1, xanchor='left', yanchor='top', font=dict(size=10, color="white")
    ),
    margin=dict(r=1, l=1, b=1, t=1),
    paper_bgcolor='black',
    plot_bgcolor='black'
)

# 📊 Vis plottet
fig.show()
